In [0]:
dbutils.widgets.text("Blob_account", "")
Blob_account = dbutils.widgets.get("Blob_account")

dbutils.widgets.text("Secret_Scope", "")
secret_scope = dbutils.widgets.get("Secret_Scope")

dbutils.widgets.text("Blob_Secret_Key", "")
Blob_Secret_Key = dbutils.widgets.get("Blob_Secret_Key")

dbutils.widgets.text("Config_file", "")
Config_file = dbutils.widgets.get("Config_file")

print(f"Blob_account : {Blob_account}")
print(f"Secret_Scope : {secret_scope}")
print(f"Blob_Secret_Key : {Blob_Secret_Key}")
print(f"Config_file : {Config_file}")

In [0]:
spark.conf.set(f"fs.azure.account.key.{Blob_account}.blob.core.windows.net",
               dbutils.secrets.get(scope=f"{secret_scope}", key=f"{Blob_Secret_Key}"))

In [0]:
from pyspark.sql.functions import *
import json

parameter_data=spark.read.format('text').option('wholetext',True).load(Config_file)
parameter_list=parameter_data.collect()

parameter_json=json.loads(parameter_list[0]['value'])
print(parameter_json)
client_id = parameter_json['Parameters']['client_id']
app_tenant_id = parameter_json['Parameters']['app_tenant_id']
vacuum_Hrs = parameter_json['Parameters']['vacuum_Hrs']
prep_database = parameter_json['Parameters']['prep_database']
struct_database = parameter_json['Parameters']['struct_database']
emails = parameter_json['Parameters']['emails']
unity_catalog = parameter_json['Parameters']['unity_catalog']
resource_id = parameter_json['Parameters']['resource_id']
app_key = parameter_json['Parameters']['app_key']
Project_Name=parameter_json['Parameters']['Project_Name']
print(f"client_id : {client_id}")
print(f"app_tenant_id : {app_tenant_id}")
print(f"vacuum_Hrs : {vacuum_Hrs}")
print(f"prep_database : {prep_database}")
print(f"struct_database : {struct_database}")
print(f"emails : {emails}")
print(f"unity_catalog : {unity_catalog}")
print(f"resource_id : {resource_id}")
print(f"app_key : {app_key}")
print(f"Project_Name : {Project_Name}")

In [0]:
def generate_html_email(paragraph="", header="", border="solid black"):
    # Basic HTML structure with enhanced styling
    style_str = """
<!DOCTYPE html>
<html>
<head>
<style>
        body {
            font-family: Arial, sans-serif;
            margin: 20px;
        }
        p {
            font-size: 16px;
            color: #333;
        }
        h3 {
            font-size: 24px;
            color: red; 
        }
</style>
</head>
<body>
    """
    # Paragraph and header
    paragraph_str = f"<p>{paragraph}</p>" if paragraph else ""
    header_str = f"<h3>{header}</h3>" if header else ""
    
    closing_str = "</body></html>"
    # Concatenate all parts
    final_html = style_str + paragraph_str + header_str + closing_str
    return final_html



In [0]:
from O365 import Account

def send_email(receiver_emails, subject, body, client_id, app_tenant_id, resource_id, secret_scope, app_key):
  
  #passing application credentials and authenticating
  credentials = (client_id, dbutils.secrets.get(scope = secret_scope, key = app_key))
  account = Account(credentials, auth_flow_type = "credentials", tenant_id = app_tenant_id)
  
  if account.authenticate():
    print("Authenticated!")
  
  m = account.new_message(resource = resource_id)
  m.to.add(receiver_emails)
  m.subject = subject
  m.body = body
  m.send()

In [0]:
tables = [
    row['tableName']
    for row in spark.sql(f"SHOW TABLES IN {unity_catalog}.{prep_database}")
                    .filter("isTemporary == false")
                    .select("tableName")
                    .collect()
    if not row['tableName'].endswith('_stg')
]

if "crp030_ppa_ppb_report_vd" in tables:
    tables.remove("crp030_ppa_ppb_report_vd")
if "flight_attndt_pre_post_month_actvty" in tables:
    tables.remove("flight_attndt_pre_post_month_actvty")
if "flight_attndt_pre_post_seq_detl" in tables:
    tables.remove("flight_attndt_pre_post_seq_detl")
if "flight_attndt_seq_detl" in tables:
    tables.remove("flight_attndt_seq_detl")
if "flight_attndt_seq_detl_emp" in tables:
    tables.remove("flight_attndt_seq_detl_emp")
if "pilot_pre_post_month_actvty" in tables:
    tables.remove("pilot_pre_post_month_actvty")
if "pilot_seq_detl" in tables:
    tables.remove("pilot_seq_detl")
if "pilot_seq_detl_emp" in tables:
    tables.remove("pilot_seq_detl_emp")
    

print(tables)

failed_tables = []

# VACUUM and then OPTIMIZE each table with error handling
for table in tables:
    full_table_name = f"{unity_catalog}.{prep_database}.{table}"
    print(f"\nProcessing table: {full_table_name}")
    
    try:
        print(f"→ Running OPTIMIZE {full_table_name}")
        spark.sql(f"""
                  OPTIMIZE {full_table_name} """)
        print(f"→ Running VACUUM {full_table_name}")
        spark.sql(f"VACUUM {full_table_name} RETAIN {vacuum_Hrs} HOURS")
    
    except Exception as e:
        print(f"Failed to process table: {full_table_name}\n   Error: {str(e)}")
        failed_tables.append((full_table_name, str(e)))

# Print summary of failures
if failed_tables:
    email_paragraph_fail = "The following tables failed during the OPTIMIZE or VACUUM process:<br><br>"
    email_paragraph_fail += "<table border='1' cellpadding='5' cellspacing='0'>"

    for table_name, _ in failed_tables:
        email_paragraph_fail += f"<li>{table_name}</li>"

    email_paragraph_fail += "</ul>"

    email_header_fail = "Orion Tables OPTIMIZE/VACUUM Failures"
    email_content_fail = generate_html_email(paragraph=email_paragraph_fail, header=email_header_fail)
    subject_fail = f"Orion Table Optimization Failures - {Project_Name}"

    send_email(emails,subject_fail,email_content_fail,client_id,app_tenant_id,resource_id,secret_scope,app_key)

else:
    print("\n✅ All Prep tables are processed successfully.")


In [0]:
tables = [
    row['tableName']
    for row in spark.sql(f"SHOW TABLES IN {unity_catalog}.{struct_database}")
                    .filter("isTemporary == false")
                    .select("tableName")
                    .collect()
    if not row['tableName'].endswith('_stg')
]

if "crp030_ppa_ppb_data" in tables:
    tables.remove("crp030_ppa_ppb_data")
if "crp030_ppc_ppd_data" in tables:
    tables.remove("crp030_ppc_ppd_data")

print(tables)

failed_tables = []

# VACUUM and then OPTIMIZE each table with error handling
for table in tables:
    full_table_name = f"{unity_catalog}.{struct_database}.{table}"
    print(f"\nProcessing table: {full_table_name}")
    
    try:
        print(f"→ Running OPTIMIZE {full_table_name} ")
        spark.sql(f"""OPTIMIZE {full_table_name}""")
        print(f"→ Running VACUUM {full_table_name}")
        spark.sql(f"VACUUM {full_table_name} RETAIN {vacuum_Hrs} HOURS")

    except Exception as e:
        print(f"Failed to process table: {full_table_name}\n   Error: {str(e)}")
        failed_tables.append((full_table_name, str(e)))

if failed_tables:
    email_paragraph_fail = "The following tables failed during the OPTIMIZE or VACUUM process:<br><br>"
    email_paragraph_fail += "<table border='1' cellpadding='5' cellspacing='0'>"

    for table_name, _ in failed_tables:
        email_paragraph_fail += f"<li>{table_name}</li>"

    email_paragraph_fail += "</ul>"

    email_header_fail = "Orion Table OPTIMIZE/VACUUM Failures"
    email_content_fail = generate_html_email(paragraph=email_paragraph_fail, header=email_header_fail)
    subject_fail = f"Orion Table Optimization Failures - {Project_Name}"

    send_email(emails,subject_fail,email_content_fail,client_id,app_tenant_id,resource_id,secret_scope,app_key)

else:
    print("\n✅ All struct tables are processed successfully.")
